In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import dash
import openpyxl
import os
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

##Iniciando o processo dados
navegador.get('https://statusinvest.com.br/acoes/eua/busca-avancada')
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[1]/div/div/div/button[2]').click()
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div[2]/a/span').click()
time.sleep(5)

#Fecha o navegador
navegador.quit()

## Renomeia o arquivo e move para pasta desejada
source = '/Users/scheiner/Downloads/statusinvest-busca-avancada.csv'
dest = '/Users/scheiner/Documents/GitHub/StatusInvest/international.csv'
os.rename(source, dest) 

print("Source path renamed to destination path successfully.")

Source path renamed to destination path successfully.


In [3]:
intl = pd.read_csv('international.csv', sep=';',  decimal=',', thousands='.', encoding='ISO-8859-1' )

intl['P/L_IDEAL'] = intl[' VPA'] / intl[' LPA']
intl['UP/DOWNSIDE'] = (intl[' VPA'] / intl['PRECO'])-1
    
value_intl = intl.loc[:, ['TICKER', 'PRECO', ' VPA' , 'DY', ' LPA', 'P/L', 'P/L_IDEAL', 'UP/DOWNSIDE', 'ROE', 
                           'P/VP', 'EV/EBIT' ,'LIQ. CORRENTE', 'MARG. LIQUIDA', 'DIV. LIQ. / PATRI.']]

value_intl.fillna(value=0, inplace=True)
value_intl.rename(columns={'LIQ. CORRENTE': "LIQ.CORR" }, inplace=True)
value_intl.rename(columns={'MARG. LIQUIDA': "MARG.LIQ" }, inplace=True)
value_intl.rename(columns={'DIV. LIQ. / PATRI.': "DIV.LIQ/PL" }, inplace=True)
    
    
# value_intl = value_intl.loc[value_intl['DY'] >= 9].loc[
#     value_intl['P/L'] >= 0.01].loc[value_intl['P/L'] < 10].loc[
#     value_intl['ROE'] >= 6].loc[
#     value_intl['EV/EBIT'] >= 0.01].loc[value_intl['EV/EBIT'] <= 10].loc[
#     value_intl['LIQ.CORR'] >= 0.5]

value_intl

,TICKER,PRECO,VPA,DY,LPA,P/L,P/L_IDEAL,UP/DOWNSIDE,ROE,P/VP,EV/EBIT,LIQ.CORR,MARG.LIQ,DIV.LIQ/PL
0,A,113.93,19.76,0.78,4.62,24.54,4.277056,-0.826560,23.37,5.73,20.10,2.37,19.19,0.27
1,AA,29.61,27.13,1.35,-8.26,-3.59,-3.284504,-0.083755,-30.44,1.09,-6.11,1.80,-13.56,0.35
2,AACG,1.18,0.72,0.00,-0.33,-3.60,-2.181818,-0.389831,-45.26,1.63,-3.09,0.23,-32.34,-0.27
3,AACQ,9.99,0.06,0.00,-0.11,-93.43,-0.545455,-0.993994,-193.66,180.94,-27.74,3.56,0.00,-0.07
4,AACQU,11.05,0.06,0.00,-0.11,-103.35,-0.545455,-0.994570,-193.66,200.14,-27.74,3.56,0.00,-0.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6169,ZUO,8.81,0.76,0.00,-1.41,-6.24,-0.539007,-0.913734,-187.20,11.69,-6.25,1.69,-47.81,-1.78
6170,ZVO,0.09,0.19,0.00,-1.20,-0.08,-0.158333,1.111111,-619.73,0.47,0.59,0.95,-17.80,-4.11
6171,ZYME,6.86,6.62,0.00,2.50,2.74,2.648000,-0.034985,37.71,1.03,0.66,4.49,37.58,-0.76
6172,ZYNE,1.33,0.54,0.00,-0.70,-1.91,-0.771429,-0.593985,-129.50,2.47,-0.95,3.43,0.00,-1.24


In [4]:
#Instalando o driver
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

ticker = value_intl['TICKER']
list_intl = ticker.to_list()

indicators_lis_intl = []
for intl in list_intl:

    #Criando url
    url = f"https://statusinvest.com.br/acoes/eua/{intl}"

    
    try:
        #Abrindo o navegador
        navegador.get(url)

        time.sleep(0.25)
    
        #Coletando infos
        NAME = navegador.find_element(By.XPATH, '//*[@id="main-header"]/div[2]/div/div[1]/div[1]/h1/small').text
        MIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[1]/div[2]/div/div[2]/div/span[2]').text
        MAX_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[1]/div[3]/div/div[2]/div/span[2]').text
        MIN_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[1]/div[2]/div/div[1]/strong').text
        MAX_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[1]/div[3]/div/div[1]/strong').text
        GAIN_12M = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[1]/div[5]/div/div[1]/strong').text
        GAIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[1]/div[5]/div/div[1]/strong').text
        ORIGEM = navegador.find_element(By.XPATH, '//*[@id="company-section"]/div[1]/div[3]/div/div[2]/div/div/div/strong/span').text
        SEGMENTO = navegador.find_element(By.XPATH, '//*[@id="company-section"]/div[1]/div[4]/div[2]/div[1]/div/div/div/a/strong').text
        
        

        #Criando dict
        dicionario = {  "intl": intl,
                        "NAME": NAME,
                        "MIN_MES": MIN_MES, 
                        "MAX_MES": MAX_MES, 
                        "MIN_52S": MIN_52S, 
                        "MAX_52S": MAX_52S, 
                        "VAL_12M": GAIN_12M,
                        "VAL_MES": GAIN_MES,
                        "ORIGEM": ORIGEM,
                        "SEGMENTO": SEGMENTO
                        
                        }

        #Criando lista de dicts
        indicators_lis_intl.append(dicionario)
    except Exception as e:
        print(e)

#Fecha o navegador
navegador.quit()

In [84]:
list_intl = pd.DataFrame.from_dict(indicators_lis_intl)

list_intl = list_intl.replace('-', '')
list_intl = list_intl.replace('-%', '')
list_intl['MIN_MES'] = (list_intl['MIN_MES']).str.lstrip('$')
list_intl['MAX_MES'] = (list_intl['MAX_MES']).str.lstrip('$')
list_intl['MIN_MES'] = (list_intl['MIN_MES']).str.strip('- ')
list_intl['MAX_MES'] = (list_intl['MAX_MES']).str.strip('- ')
list_intl['VAL_12M'] = (list_intl['VAL_12M']).str.rstrip('%')
list_intl['VAL_MES'] = (list_intl['VAL_MES']).str.rstrip('%')

list_intl.to_csv('list_intl.csv')

list_intl = pd.read_csv("list_intl.csv", sep=',', decimal=',', thousands='.', encoding='ISO-8859-1' )

list_intl.fillna(value=0, inplace=True)
list_intl = list_intl.drop(['Unnamed: 0'],axis=1)

value_intl.rename(columns={'TICKER': "TICKERS" }, inplace=True)

list_intl.rename(columns={'intl': "TICKERS" }, inplace=True)

invest_intl = pd.merge(value_intl, list_intl, on='TICKERS')

invest_intl = invest_intl[['TICKERS', 'NAME', 'PRECO', 'MIN_MES', 'MAX_MES', 'VAL_MES', 'MIN_52S', 'MAX_52S',  'VAL_12M', 
                       ' VPA', 'DY', ' LPA', 'P/L' ,'P/L_IDEAL', 'UP/DOWNSIDE', 'ROE', 'P/VP', 'EV/EBIT', 
                       'LIQ.CORR', 'MARG.LIQ', 'DIV.LIQ/PL', 'ORIGEM', 'SEGMENTO']]

invest_intl = invest_intl.loc[invest_intl['DY'] >= 0].loc[
    invest_intl['P/L'] >= 0.01].loc[invest_intl['P/L'] < 10].loc[
    invest_intl['ROE'] >= 10].loc[
    invest_intl['EV/EBIT'] >= 0.01].loc[invest_intl['EV/EBIT'] <= 10].loc[
    invest_intl['LIQ.CORR'] >= 0.5].loc[
    invest_intl['UP/DOWNSIDE'] >= 0.1]

#invest_intl[(invest_intl['TICKERS'] == 'MSFT')]
#invest_intl.loc[invest_intl['SEGMENTO'] == 'Technology'].loc[invest_intl['ORIGEM'] == 'NASDAQ']
# 
invest_intl


,TICKERS,NAME,PRECO,MIN_MES,MAX_MES,VAL_MES,MIN_52S,MAX_52S,VAL_12M,VPA,...,P/L_IDEAL,UP/DOWNSIDE,ROE,P/VP,EV/EBIT,LIQ.CORR,MARG.LIQ,DIV.LIQ/PL,ORIGEM,SEGMENTO
295,AMPY,AMPLIFY ENERGY CORP.,7.45,6.60,7.50,-2.61,5.66,10.21,-2.61,9.17,...,0.814387,0.230872,122.81,0.82,2.01,0.87,116.64,0.33,NYSE,Energy
683,BEST,BEST INC.,2.51,2.37,2.90,136.79,0.50,3.01,136.79,10.16,...,1.946360,3.047809,51.40,0.25,2.05,0.90,44.90,-1.61,NYSE,Transportation
703,BGFV,BIG 5 SPORTING GOODS CORPORATION,7.14,6.86,7.65,-40.65,6.86,13.56,-40.65,12.11,...,6.147208,0.696078,16.29,0.58,2.19,1.61,4.30,-0.11,NASDAQ,Consumer Services
713,BH,BIGLARI HOLDINGS INC.,178.47,178.43,186.76,45.68,115.60,212.62,45.68,198.63,...,5.660587,0.112960,17.67,0.90,3.65,1.26,29.10,-0.02,NYSE,Consumer Services
855,BRID,BRIDGFORD FOODS CORPORATION,11.01,10.61,11.67,-21.30,10.61,15.00,-21.30,14.04,...,2.830645,0.275204,35.35,0.78,1.40,4.18,17.39,-0.09,NASDAQ,Consumer Non-Durables
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6059,X,UNITED STATES STEEL CORPORATION,30.48,30.22,31.14,51.38,18.12,32.15,51.38,48.10,...,8.003328,0.578084,12.49,0.64,5.20,1.92,7.05,0.11,NYSE,Basic Industries
6064,XCUR,EXICURE INC.,0.83,0.81,0.96,-59.90,0.71,2.07,-59.90,1.12,...,3.200000,0.349398,31.58,0.77,1.56,2.56,12.88,-0.24,NASDAQ,Health Care
6098,XYF,X FINANCIAL,4.45,4.15,4.49,112.80,1.62,5.00,112.80,13.52,...,5.656904,2.038202,17.71,0.33,0.24,1.89,20.98,-0.25,NYSE,0
6116,YPF,YPF SOCIEDAD ANONIMA,12.83,11.99,14.20,93.40,6.21,15.67,93.40,25.42,...,6.468193,0.981294,15.46,0.51,7.34,1.15,9.72,0.57,NYSE,Energy
